In [68]:
# Generic packages
import numpy as np
import requests
import pandas as pd
import copy
import datetime

# Basketball package
from espn_api.basketball import League

# FB package
import fbchat

# email
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import smtplib

# set up the SMTP server
s = smtplib.SMTP(host='smtp.gmail.com', port=587)
s.starttls()
s.login('dailyfantasybot@gmail.com', 'fantasyfellas')

(235, b'2.7.0 Accepted')

In [70]:
# constant variables
SWID = '{C53D70DA-4087-40A6-B17E-92381F80A011}'
ESPN_S2 = "AEBwkPP4TglBo0Bevp1ODtJK%2B1xBcaq9LCvVPpF02qRu9DZP4GAk08HeGxTFpGhjH%2BBexBlqhfpo6Tad%2FNnpTPEX68CfLPSVoeVAUbo81qW7qo5PB2U0CFc8JzbrfBSI9pAgC98c9S6b2i30s1S1gEjt6ZNPuBBCngtefS%2BOQCKeBDuUsboM1dBVpi9YmHALaUGR0qV7Z8aVBT2vDM0bIabR1y%2F61Tx40K8o%2FIj54aW0REiij9B0MZxWLF%2FgxeBx84D9tfORhbxBsIOSMLUZfgsE"
LEAGUE = 55416621
YEAR = 2022

FB_USER = 'tobyprofitt@hotmail.com'
FB_PASS = '788drummond'
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
USER_AGENT2 = 'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko'

PRO_TEAM_MAPPING =  {'ATL':1, 'BOS':2, 'NOP':3, 'CHI':4, 'CLE':5, 'DAL':6, 'DEN':7, 'DET':8, 'GSW':9, 'HOU':10, 'IND':11, 'LAC':12, 'LAL':13, 'MIA':14, 'MIL':15, 
'MIN':16, 'BKN':17, 'NYK':18, 'ORL':19, 'PHL':20, 'PHO':21, 'POR':22, 'SAC':23, 'SAS':24, 'OKC':25, 'UTA':26, 'WAS':27, 'TOR':28, 'MEM':29, 'CHA':30}
PRO_TEAM_MAPPING_COPY = copy.copy(PRO_TEAM_MAPPING)
for k, v in PRO_TEAM_MAPPING_COPY.items():
    PRO_TEAM_MAPPING[v] = k

email_jack   = 'Jack\'s email'
email_toby   = 'tobyprofitt1@gmail.com'
email_cam    = 'Cam\'s email'
email_nick   = 0
email_markey = 0
email_mish   = 0

In [3]:
# Gets the current points standings
league = League(league_id=LEAGUE, year=YEAR, espn_s2 = ESPN_S2, swid = SWID)

Jack, Toby, Nick, Mish, Cam, Markey = league.teams

EMAILS = {Jack.team_id:email_jack, Toby.team_id:email_toby, Cam.team_id:email_cam, 
        Nick.team_id:email_nick, Markey.team_id:email_markey, Mish.team_id:email_mish}

stats = [Jack.stats, Toby.stats, Nick.stats, Mish.stats, Cam.stats, Markey.stats]
leaderboard = pd.DataFrame(stats)
leaderboard.index = ['Jack', 'Toby', 'Nick', 'Mish', 'Cam', 'Markey']
gp = leaderboard.GP
totals = leaderboard
points = leaderboard.drop(labels= ['GP', 'FGM', 'FGA', 'FTM', 'FTA'], axis=1).rank()
points = pd.concat([points, points.sum(axis=1)], axis=1)
points = points.rename(columns = {0: 'TOT'})
points = points.sort_values(by=['TOT'], ascending=False)
points = pd.concat([points, gp], axis = 1)
points

,PTS,BLK,STL,AST,REB,3PTM,FG%,FT%,TOT,GP
Jack,6.0,5.0,3.0,5.0,5.0,4.0,2.0,3.0,33.0,341.0
Markey,1.0,6.0,6.0,6.0,6.0,1.0,6.0,1.0,33.0,316.0
Toby,5.0,3.0,4.0,1.0,4.0,5.0,4.0,6.0,32.0,335.0
Nick,4.0,2.0,5.0,4.0,1.0,6.0,1.0,4.0,27.0,322.0
Cam,3.0,4.0,2.0,3.0,2.0,3.0,5.0,5.0,27.0,312.0
Mish,2.0,1.0,1.0,2.0,3.0,2.0,3.0,2.0,16.0,297.0


In [9]:
for team in league.teams:
    print(team.team_id, team.team_name)


1 The Cheese
2 Drummond st Demons
7 Ballarat Timberwolves
8 Sydney Mish
9 Brunswick Stink
10 Canberra Marks


In [5]:
# Investigating 'injury report'
for team in league.teams:
    print("Injury report for:", str(team)[5:-1], end='\n\n')
    TO_EMAIL = EMAILS[team.team_id]
    for player in team.roster:
        if player.injuryStatus != "ACTIVE":
            extraMessage = ''
            if player.lineupSlot not in ['BE', 'IR']:
                extraMessage = '... Put them in your IR or bench if you can!'
            print(str(player)[7:-1] + ' (' + str(player.lineupSlot) + ') ' + 'is', player.injuryStatus, extraMessage)
    print('')

Injury report for: The Cheese

Luka Doncic (BE) is OUT 
LeBron James (SF) is DAY_TO_DAY ... Put them in your IR or bench if you can!
Shai Gilgeous-Alexander (PG) is DAY_TO_DAY ... Put them in your IR or bench if you can!
Julius Randle (F) is OUT ... Put them in your IR or bench if you can!
Mo Bamba (IR) is OUT 
Fred VanVleet (BE) is OUT 
Malcolm Brogdon (BE) is OUT 
Kyle Lowry (IR) is OUT 
Jayson Tatum (PF) is OUT ... Put them in your IR or bench if you can!

Injury report for: Drummond st Demons

Jimmy Butler (UT) is DAY_TO_DAY ... Put them in your IR or bench if you can!
Anthony Davis (IR) is OUT 
Deandre Ayton (BE) is OUT 
Klay Thompson (IR) is OUT 
Jarrett Allen (BE) is OUT 
Josh Hart (F) is DAY_TO_DAY ... Put them in your IR or bench if you can!

Injury report for: Ballarat Timberwolves

Karl-Anthony Towns (C) is OUT ... Put them in your IR or bench if you can!
Anthony Edwards (UT) is OUT ... Put them in your IR or bench if you can!
D'Angelo Russell (UT) is OUT ... Put them in you

In [31]:
dame = Toby.roster[0]
dame.lineupSlot
dame.proTeam

{13: (9, 1634695200000),
 15: (17, 1634686200000),
 17: (15, 1634686200000),
 9: (13, 1634695200000)}

In [89]:
# Get nba schedule for a given day
import datetime

first_day = datetime.date(2021, 10, 19)
today = datetime.date.today()
diff = today - first_day
gameday_id = diff.days
# http://site.api.espn.com/apis/site/v2/sports/basketball/nba/teams
todays_games = league._get_pro_schedule(gameday_id)

for player in Toby.roster:
    print(player.name, player.lineupSlot, end = '... ')
    team_id = PRO_TEAM_MAPPING[player.proTeam]
    if team_id in todays_games.keys():
        opponent_id = todays_games[team_id][0]
        opponent_name = PRO_TEAM_MAPPING[opponent_id]
        print('They are playing', opponent_name, 'today!')
    else:
        print('')


Damian Lillard PG... They are playing ATL today!
Stephen Curry G... They are playing MIA today!
Jimmy Butler UT... They are playing GSW today!
Anthony Davis IR... 
Deandre Ayton BE... 
Tobias Harris PF... They are playing HOU today!
Khris Middleton SF... They are playing DET today!
Klay Thompson IR... They are playing MIA today!
Gordon Hayward F... They are playing WAS today!
Domantas Sabonis BE... 
Zach LaVine SG... They are playing ORL today!
Jarrett Allen BE... 
Desmond Bane UT... They are playing BKN today!
Marcus Morris Sr. UT... They are playing MIN today!
Jaren Jackson Jr. C... They are playing BKN today!


In [44]:
# Get nba schedule for a given day
import datetime

first_day = datetime.date(2021, 10, 19)
today = datetime.date.today()
diff = today - first_day
gameday_id = diff.days
# http://site.api.espn.com/apis/site/v2/sports/basketball/nba/teams
league._get_pro_schedule(gameday_id)

{12: (16, 1641267000000),
 14: (9, 1641265200000),
 15: (8, 1641258000000),
 16: (12, 1641267000000),
 17: (29, 1641256200000),
 19: (4, 1641258000000),
 20: (10, 1641254400000),
 22: (1, 1641265200000),
 26: (3, 1641258000000),
 27: (30, 1641254400000),
 29: (17, 1641256200000),
 30: (27, 1641254400000),
 1: (22, 1641265200000),
 3: (26, 1641258000000),
 4: (19, 1641258000000),
 6: (7, 1641259800000),
 7: (6, 1641259800000),
 8: (15, 1641258000000),
 9: (14, 1641265200000),
 10: (20, 1641254400000)}

Next part is testing out how to email

In [ ]:
def send_mail(to_mail, message):
    from_mail = 'dailyfantasybot@gmail.com'
    #to_mail = 'tobyprofitt1@gmail.com'

    msg = MIMEMultipart()

    # setup the parameters of the message
    msg['From']=from_mail
    msg['To']=to_mail
    msg['Subject']="This is TEST"

    # add in the message body
    msg.attach(MIMEText(message, 'plain'))

    # send the message via the server set up earlier.
    s.send_message(msg)
        
    del msg